## Рекурсивные функции

`Рекурсивная функция` - это функция, которая вызывает сама себя, и при каждом очередном вызове использует данные, созданные во время предыдущего вызова. В программировании есть ряд задач, которые проще (но не всегда эффективнее) решаются с помощью рекурсии. Чтобы разобраться в принципе работы рекурсивных функций, нужно понять (в самых общих чертах) концепцию `стека` вызовов.

`Стек` – это структура данных LIFO (last in, first out): информация последовательно добавляется в «стопку» , каждый новый объект помещается поверх предыдущего, а извлекаются объекты в обратном порядке, – начиная с верхнего. Работу стека отлично иллюстрирует добавление данных в список с помощью `append` и извлечение информации посредством `pop`:

In [1]:
stack = []
for i in range(1, 6):
    stack.append(f'{i}-й элемент')
    print(f'+ {i}-й элемент добавлен')
    for i in stack:
        print(i, end=" ")
print()
print()
for i in range(len(stack)):
    print('В стеке: ', end=" ")
    for i in stack:
        print(i, end=" ")
    print(f'\n{stack.pop()} удален из стека')

+ 1-й элемент добавлен
1-й элемент + 2-й элемент добавлен
1-й элемент 2-й элемент + 3-й элемент добавлен
1-й элемент 2-й элемент 3-й элемент + 4-й элемент добавлен
1-й элемент 2-й элемент 3-й элемент 4-й элемент + 5-й элемент добавлен
1-й элемент 2-й элемент 3-й элемент 4-й элемент 5-й элемент 

В стеке:  1-й элемент 2-й элемент 3-й элемент 4-й элемент 5-й элемент 
5-й элемент удален из стека
В стеке:  1-й элемент 2-й элемент 3-й элемент 4-й элемент 
4-й элемент удален из стека
В стеке:  1-й элемент 2-й элемент 3-й элемент 
3-й элемент удален из стека
В стеке:  1-й элемент 2-й элемент 
2-й элемент удален из стека
В стеке:  1-й элемент 
1-й элемент удален из стека


`Стек вызовов`, в свою очередь, – это область памяти, в которой выполняются функции. При каждом вызове функции создается фрейм – фрагмент памяти, – в котором содержится:

- информация о текущем состоянии выполнения функции;
- значения всех переменных, которые функция получила для обработки;
- локальные данные, созданные во время очередного вызова;
- сведения о строке программы, к которой нужно вернуться после выполнения функции.

Фреймы помещаются в стек вызовов, как уже было показано в примере выше, и удаляются точно так же, сверху вниз. Рекурсивные функции при каждом новом вызове используют данные, созданные во время работы предыдущего вызова.


Неверное же использование рекурсии приводит к переполнению стека (stack overflow). Популярный сайт StackOverflow назван как раз в честь этой ошибки.

Переполнить стек в опытных целях можно с помощью простейшей рекурсивной функции, которая бесконечно вызывает сама себя, но не возвращает никаких данных и не содержит никакого условия для прекращения своей работы:

In [3]:
def recursive():
    recursive()

recursive()

RecursionError: maximum recursion depth exceeded

Интерпретатор Python автоматически отслеживает переполнение стека и после 3000 бесплодных вызовов завершает работу подобных функций с ошибкой


При желании лимит на глубину рекурсии можно увеличить, но сделать его бесконечным, разумеется, нельзя – даже самый внушительный объем оперативной памяти в итоге окажется переполненным:

In [4]:
from sys import getrecursionlimit
from sys import setrecursionlimit
print(getrecursionlimit()) # выводит лимит по умолчанию
setrecursionlimit(5000) # увеличивает лимит до 2000 вызовов
print(getrecursionlimit())# выводит новый лимит

3000
5000


Чтобы стек вызовов не переполнялся, в каждой рекурсивной функции всегда должны быть предусмотрены два случая:

- Граничный, при котором функция завершает работу и возвращает данные в основную программу.
- Рекурсивный, при котором функция продолжает вызывать себя.

Вот пример простейшей рекурсивной функции, в которой учтены оба случая:

In [5]:
def greetings(word):
    print(word)
    
    if len(word) == 0:
        return
    else:
        greetings(word[:-1])

greetings("Hello world!")

Hello world!
Hello world
Hello worl
Hello wor
Hello wo
Hello w
Hello 
Hello
Hell
Hel
He
H



In [6]:
greetings("Nikita")

Nikita
Nikit
Niki
Nik
Ni
N



In [7]:
def greetings(word):
    print(word)
    
    if len(word) >0:
        greetings(word[:-1])

greetings("Hello world!")

Hello world!
Hello world
Hello worl
Hello wor
Hello wo
Hello w
Hello 
Hello
Hell
Hel
He
H



## Скорость выполнения: итерация vs рекурсия


Рекурсивные функции работают медленнее обычных, поэтому их стоит применять только тогда, когда решить задачу без рекурсии сложно. Вот сравнение времени выполнения двух функций, рекурсивной `fib_recursive(n)` и обычной `fib_iter(n)`, решающих одну и ту же задачу – вычисление последовательности Фибоначчи:

In [8]:
from timeit import timeit

def fib_iter(n):
    if n == 1:
        return [1]
    if n == 2:
        return [1, 1]
    fibs = [1, 1]
    for _ in range(2, n):
        fibs.append(fibs[-1] + fibs[-2])
    return fibs

setup_code_iter = 'from __main__ import fib_iter'
stmt_iter = 'fib_iter(15)'
print('Время выполнения итеративной функции: ', timeit(setup=setup_code_iter, stmt=stmt_iter, number=20000))

def fib_recursive(n):
    if(n <= 1):
        return n
    else:
        return(fib_recursive(n-1) + fib_recursive(n-2))
    
setup_code_rec = 'from __main__ import fib_recursive'
stmt_rec = 'fib_recursive(15)'
print('Время выполнения рекурсивной функции: ', timeit(setup=setup_code_rec, stmt=stmt_rec, number=20000))

Время выполнения итеративной функции:  0.1247612499864772
Время выполнения рекурсивной функции:  2.494186917087063


В приведенном выше примере в обычной функции используется цикл for. Цикл выполняет итерацию (перебор), причем справляется с задачей гораздо быстрее, чем рекурсивная функция, поскольку рекурсия совершает множество повторных вызовов, и с увеличением числа элементов последовательности количество повторов растет лавинообразно:

![fibb](fib.png)

Итерацию можно назвать противоположностью рекурсии. Всегда, когда задачу можно решить итерацией (либо итерацией с использованием стека), следует делать выбор в пользу цикла for или while вместо рекурсии.

Задание 1
Напишите функцию для вычисления факториала числа. Решите задачу двумя способами – итеративным и рекурсивным.

Примечание для рекурсивного решения: предположим, нам нужно вычислить 5! Факториал 5 равен: 5 х 4 х 3 х 2 х 1. Факториал 4: 4 х 3 х 2 х 1, факториал 3: 3 х 2 х 1, факториал 2: 2 х 1, и факториал 1 равен 1. Очевидно, что 5! = 5 x 4!, 4! = 4 x 3!, 3! = 3 x 2! и так далее до граничного случая 1! = 1, то есть каждый последующий факториал включает в себя определение предыдущего.

Решение 1 – итеративное:

In [9]:
def fact_iter(n):
    factorial = 1
    for i in range(1, n + 1):
        factorial *= i
    return factorial
print(fact_iter(int(input())))

5
120


Решение 2 – рекурсивное:

In [10]:
def fact_recursive(n):
    if n == 1: # граничный случай
        return 1
    else: # рекурсивный случай
        return n * fact_recursive(n - 1)
print(fact_recursive(int(input())))

5
120


Задание 2
Напишите программу для возведения числа n в степень m. Решите задачу двумя способами – итеративным и рекурсивным.

Примечание для рекурсивного решения: предположим, что нужно возвести число 5 в степень 6. Свойства степени позволяют разбить процесс на более мелкие операции и представить выражение 5 ** 6 в виде (5 ** 3) ** 2. Этот подход работает в том случае, если степень представляет собой четное число. Если степень нечетная, следует воспользоваться другим свойством: (n ** m) x n = n ** (m + 1). Поскольку может ввести как четное, так и нечетное значение m, в функции должны быть два рекурсивных случая. В качестве граничного случая используется еще одно свойство степени: n ** 1 = n.

Пример ввода:

        
12

8

    
Вывод:

        
429981696

Решение 1 – итеративное:

In [11]:
def pow_iter(n, m):
    res = 1
    for i in range(m):
        res *= n
    return res
n, m = int(input()), int(input())
print(pow_iter(n, m))

12
8
429981696


Решение 2 – рекурсивное:

In [12]:
def pow_recursive(n, m):
    if m == 1: # граничный случай
        return n
    elif m % 2 == 0: # четный рекурсивный случай
        res = pow_recursive(n, m // 2)
        return res * res
    else: # нечетный рекурсивный случай
        res = pow_recursive(n, m // 2)
        return res * res * n
n, m = int(input()), int(input())
print(pow_recursive(n, m))

12
8
429981696


Задание 3
Напишите итеративную и рекурсивную функции для вычисления суммы n первых членов геометрической прогрессии:

<!-- ![task](task3.png) -->

S = 1/2 + 1/(2^2) + 1/(2^3) + ... + + 1/(2^(n-1)) + 1/(2^n)

Пример ввода:

        
9

    
Вывод:

        
1.99609375

Решение 1: итеративное

In [14]:
def geometric_iter(n):
    res = 0
    for i in range(n):
        res += 1 / 2 ** i
    return res    
print(geometric_iter(int(input())))

9
1.99609375


Решение 2: рекурсивное

In [13]:
def geometric_rec(n):
    if n < 0: # граничный случай
        return 0
    else: # рекурсивный случай
        return 1/2 ** n + geometric_rec(n-1)

print(geometric_rec(int(input())))

9
1.998046875


Примечание: если знаменатель не равен 1, задачу можно решить с помощью формулы суммы n первых членов геометрической прогрессии:

In [15]:
b, q, n = 1, 0.5, int(input())
print(b * (1 - q ** n) / (1 - q))

9
1.99609375


Задание 4
Напишите рекурсивную функцию, которая определяет, является ли введенная пользователем строка палиндромом.


Пример ввода:

        
Лёша на полке клопа нашёл

    
Вывод:

        
True

    


Рекурсивное решение

In [16]:
def palindrome(my_str):
    if len(my_str) == 0 or len(my_str) == 1: # граничный случай
        return True
    else: # рекурсивный случай
        head = my_str[0]
        middle = my_str[1:-1]
        end = my_str[-1]
        return head == end and palindrome(middle)

st = [i.lower() for i in input() if i.isalpha()]
print((palindrome(st)))

Лёша на полке клопа нашёл
True


Решение без рекурсии

In [21]:
st = [i.lower() for i in input() if i.isalpha()]
print(st == st[::-1])

Лёша на полке клопа нашёл
True
